# Hare or Rabbit?
###Projeto desenvolvido em python para classificar se dado animal é um coelho ou uma lebre

In [ ]:
# Baixando os arquivos de treinamento e teste

!git clone https://github.com/leticiap/hare-or-rabbit.git

In [ ]:
# Imports

import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import os

from time import time
from torchvision import transforms
from torch import nn, optim
from datetime import datetime
from tqdm import tqdm
from skimage.io import imread
from skimage.color import rgba2rgb
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# Extraindo as imagens para treinamento

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Start Time = ", current_time)
train = pd.read_csv('hare-or-rabbit/dataset.csv')
train.head()

train_img=[]

print ("loading")
for img_name in tqdm(train['image_name']):
    img = imread(img_name)
    # if we have 4 channels instead of 3
    if img.shape[2] != 3:
        img = rgba2rgb(img)
    img = img.astype('float32')
    train_img.append(img)

In [ ]:
# Conversão de dados, iremos separar 70% das imagens para treino e 30% das imagens para validação (teste)

print ("Converting")

img_size = 150
channels = 3
train_x = np.array(train_img)
train_y = train['label'].values

# criando set de validação
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.3)
print((train_x.shape, train_y.shape), (val_x.shape, val_y.shape))

# convertendo as imagens de treino para o torch
train_x = train_x.reshape(train_x.shape[0], 3, img_size, img_size)
train_x  = torch.from_numpy(train_x)

# convertendo alvos para o torch
train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)

# convertendo imagens de validação para o torch
val_x = val_x.reshape(val_x.shape[0], 3, img_size, img_size)
val_x  = torch.from_numpy(val_x)

# convertendo alvos para o torch
val_y = val_y.astype(int);
val_y = torch.from_numpy(val_y)

tensor_x = torch.Tensor(train_x) # transformar para o torch tensor
tensor_y = torch.Tensor(train_y)

my_dataset = TensorDataset(tensor_x,tensor_y) # criando o dataset de treino
trainloader = DataLoader(my_dataset) # criando o set de treino

tensor_x = torch.Tensor(val_x) # transformando para o torch tensor
tensor_y = torch.Tensor(val_y)

my_dataset = TensorDataset(tensor_x,tensor_y) # criando o dataset de teste
testloader = DataLoader(my_dataset) # criando o set de teste

Converting
((158, 150, 150, 3), (158,)) ((69, 150, 150, 3), (69,))


In [ ]:
# definição da CNN

class Modelo(nn.Module):
  def __init__(self):
    super(Modelo, self).__init__()
    self.linear1 = nn.Linear(img_size*img_size * channels, 4096) # entrada do tamanho completo da imagem, que se conecta com 4096 neurônios
    self.linear2 = nn.Linear(4096, 1024) # camada interna 1,  4096 -> 1024 neurônios, dando o resultado
    self.linear3 = nn.Linear(1024, 64) # camada interna 2, 1024 -> 64 neurônios, dando o resultado
    self.linear4 = nn.Linear(64, 2) # camada interna 3, 64 -> 2 neurônios, dando o resultado

  def forward(self, X):
    X = F.relu(self.linear1(X))
    X = F.relu(self.linear2(X))
    X = F.relu(self.linear3(X))
    X = self.linear4(X)
    return F.log_softmax(X, dim=1)

In [ ]:
# Função de treinamento

def treino(modelo, trainloader, device):
  otimizador = optim.SGD(modelo.parameters(), lr=0.01, momentum=0.5)
  start = time()

  criterio = nn.NLLLoss()
  epochs = 10
  modelo.train()

  for epoch in range(epochs):
    perda_acumulada = 0

    for imagens, etiquetas in trainloader:
      imagens = imagens.view(imagens.shape[0], -1)
      otimizador.zero_grad()

      output = modelo(imagens.to(device))
      perda_instantanea = criterio(output, etiquetas.to(device))

      perda_instantanea.backward()

      otimizador.step()

      perda_acumulada += perda_instantanea.item()

  print(f"Training time {(time()-start)/60}")


In [ ]:
# Função de validação
def validacao(modelo, valloader, device):
  corretos, total = 0, 0
  for imagens, etiquetas in valloader:
    for i in range(len(etiquetas)):
      img = imagens[i].view(1, img_size * img_size * channels)
      with torch.no_grad():
        logps = modelo(img.to(device))

      ps = torch.exp(logps)
      probab = list(ps.cpu().numpy()[0])
      etiqueta_pred = probab.index(max(probab))
      etiqueta_certa = etiquetas.numpy()[i]
      if (etiqueta_certa == etiqueta_pred):
        corretos += 1
      total += 1

  print(f"Total de imagens testadas: {total}")
  print(f"Precisão do modelo: {corretos*100/total}")

In [ ]:
# Executando todo o processo da rede (treinar e validar)

modelo = Modelo()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo.to(device)

treino(modelo, trainloader, device)
validacao(modelo, testloader, device)

Training time 34.51368735631307
Total de imagens testadas: 69
Precisão do modelo: 53.6231884057971
